## Design Cassandra database for Analyzing Songs And User Listening Sessions Data

This notebook includes the ETL pipeline a) to process the event data collected by the music streaming app, and b) create a Cassandra keyspace for the analysis team to query against the data to understand what songs users are listening to.

The event data is stored in CSV files, one for each day collected.  Files are placed in `/data/event_data/` directory.

The ETL pipeline process traverses the data directory, reads in data from each file and then write out a single CSV file, `/data/processed_events/event_datafile_new.csv` file.

Once the new file is created, its content is loaded loaded into the pandas dataframe for analysis.

Helper functions `create_table`, `populate_table`, `fetch`, `drop_table`, `pandas_factory` are created aid with data analysis:
* `create_table` - given a current session, create a table in the Cassandra keyspace with given table name and columns for the table from `table_columns` arg.  Primary keys for the table is specified with `primary_keys` arg.  The optional `clustering_key` arg is for specifying additional key for primary key and sorting
* `populate_table` - populating the table with specific fields loaded from the pandas dataframe
* `fetch` - fetch data using giving SQL query statement
* `drop_table` - delete table from keyspace as needed
* `pandas_factory` - specify panda dataframe as the data structure return query results 

Querying data is a two-steps task. 

1. you need to invoke `create_table` to create a table and populate it with data extracted from data frame,
2. create the SQL statement you want to run against the table and invoke `fetch` function to get the result of the query.

### Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
     
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/Users/ai.tran/src/data-engineering/data-modeling/etl-pipeline


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# specify the name of the file in which the extracted data is stored
processed_events_file = os.getcwd() + '/data/processed_events/event_datafile_new.csv'

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of 
# event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv 
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(processed_events_file, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

print(processed_events_file)

/Users/ai.tran/src/data-engineering/data-modeling/etl-pipeline/data/processed_events/event_datafile_new.csv


In [4]:
df = pd.read_csv(processed_events_file, header=0)
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [5]:
df.shape

(6820, 11)

### Part II. Load processed data file and create tables for analyzing songs and users' listening sessions information. 

#### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="assets/image_event_datafile_new.jpg">

#### Define functions for creating table, populating table with datasource, run query and return results as panda dataframe, and dropping table.  The pandas factory is defined to be used as the row factory for the session

In [6]:
def create_table(session, table_name, table_columns, primary_keys, clustering_keys=None):
    """ 
        This function create a table and populate it with data from a global 
        defined data source.

        A table with `table_name` is created in given keyspace with columns defined 
        from given `table_columns` list.
        
        The primary keys for table are specified in `primary_keys` and clustering_keys.
        
        This includes running `drop table` operation first to ensure existing table 
        is deleted prior to creating a new one.
    """

    if session is None or table_name is None or table_columns is None or primary_keys is None or len(primary_keys) == 0:
        print("Must provide active session, table name and columns plus data type, and one or more primary keys")
        return

    # drop existing table with same name
    drop_table(session, table_name)

    # declare table
    create_stmt = f"CREATE TABLE IF NOT EXISTS {table_name} ("

    # define columns in table with their corresponding data type
    for column, data_type in table_columns.items():
        create_stmt = create_stmt + f"{column} {data_type}, "

    # define primary keys for table
    create_stmt = create_stmt + f"PRIMARY KEY (({', '.join(primary_keys)})"

    # append clustering keys for sorting if given
    if clustering_keys is not None:
        create_stmt = create_stmt + f", {', '.join(clustering_keys)}"
    
    create_stmt = create_stmt + "))"
    print(create_stmt)  

    # create operation
    try:
        session.execute(create_stmt)
    except Exception as e:
        print(e)

    # populate the table with data
    populate_table(session, table_name, list(table_columns.keys()))


def populate_table(session, table_name, table_columns):
    """ 
        Populate the table with data extracted from global data source, df. 
        
        Notes: The global data source is used for convenient, without the need 
        to open the same csv file and create the dataframe each time. 
        
        If the data file is different for each table, it is recommended that the 
        the data file should be provided and data be loaded into new dataframe
        each time function is invoked.
    """

    if session is None or table_name is None or table_columns is None:
        print("Must provide active session, table name plus columns plus data type")
        return

    insert_stmt = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({'%s, '*(len(table_columns)-1)}%s)"
    try:
        [session.execute(insert_stmt, tuple(row)) for row in df[table_columns].to_numpy()]
    except Exception as e:
        print(e)


def fetch(session, query):
    """ Fetch data from database using given query statement """

    if session is None or query is None:
        print("Must provide active session, and query to execute")
        return

    print(f"===== Query: =======\n{query}\n")
    try:
        result = session.execute(query, timeout=None)
        dataframe = result._current_rows
        return dataframe
    except Exception as e:
        print(e)
    

def drop_table(session, table_name):
    """ Drop the given table from keyspace of given session """

    drop_stmt = f"DROP TABLE IF EXISTS {table_name}"
    try:
        session.execute(drop_stmt)
    except Exception as e:
        print(e)


def pandas_factory(colnames, rows):
    """ Define pandas dataframe as the row factory for cassandra session """
    return pd.DataFrame(rows, columns=colnames)


#### Creating a Cluster

In [7]:
# This should make a connection to a single-node Cassandra cluster instance on local machine 
# (127.0.0.1), at port 6000.  Note that the default port for cassandra database is 9042
# 
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'], 6000)

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    session.row_factory = pandas_factory
    session.default_fetch_size = None
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

#### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In order to run this query, the table must contain the following fields:
* sessionId:        int
* itemInSession:    int
* artist:           text
* song:             text
* length:           float

These fields are defined in the `table_columns` argument.

The primary key is a composite key comprised of columns that are used for querying: sessionId and itemInSession.


In [10]:
table_1 = "song_history_by_session"

## create a table for storing data
create_table(session, 
    table_name=table_1, 
    table_columns={'sessionId':'int', 'itemInSession':'int', 'artist':'text', 'song':'text', 'length':'float'}, 
    primary_keys=('sessionId', 'itemInSession')
)

## Add in the SELECT statement to verify the data was entered into the table
query = f"""
    SELECT artist, song, length
    FROM {table_1}
    WHERE sessionId=338 AND itemInSession=4"""
df1 = fetch(session, query)
df1

CREATE TABLE IF NOT EXISTS song_history_by_session (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY ((sessionId, itemInSession)))
===== Query: =======

    SELECT artist, song, length
    FROM song_history_by_session
    WHERE sessionId=338 AND itemInSession=4



,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In order to run this query, the table must contain the following fields:
* userId: int
* sessionId: int
* artist: text
* song: text
* firstName: text
* lastName: text

These fields are defined in the `table_columns` argument.

The primary key of this table is a composite key including `userId`, `sessionId`, and the clustering key `itemInSession`. 

`itemInSession` is added as clustering column for sorting the resultset.

In [11]:
table_2 = "song_history_by_user_session"

create_table(session, 
    table_name=table_2, 
    table_columns={'sessionId':'int', 'userId':'int', 'itemInSession':'int', 'artist':'text', 'song':'text', 'firstName':'text', 'lastName':'text'}, 
    primary_keys=['sessionId', 'userId'],
    clustering_keys=['itemInSession']
)

query = f"""
    SELECT artist, song, firstName, lastName
    FROM {table_2}
    WHERE userId=10 AND sessionId=182 
    ORDER BY itemInSession"""
df2 = fetch(session, query)
df2

CREATE TABLE IF NOT EXISTS song_history_by_user_session (sessionId int, userId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((sessionId, userId), itemInSession))
===== Query: =======

    SELECT artist, song, firstName, lastName
    FROM song_history_by_user_session
    WHERE userId=10 AND sessionId=182 
    ORDER BY itemInSession



,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In order to run this query, the table must contain the following fields:
* song: text
* userId: int
* firstName: text
* lastName: text

These fields are defined in the table_columns argument.

The primary key (PK) for this table is a composite key comprised of `song` and `userId` columns.  
`song` is used as PK because it is used for looking up the specific song.  `userId` column is added as the second key
to ensure uniqueness of each row in the resultset.


In [12]:
table_3 = "song_listened_by_users"

create_table(session, 
    table_name=table_3, 
    table_columns={'song':'text', 'userId':'int', 'firstName':'text', 'lastName':'text'}, 
    primary_keys=['song'],
    clustering_keys=['userId']
)

query = f"""
    SELECT firstName, lastName 
    FROM {table_3} 
    WHERE song='All Hands Against His Own'"""
df3 = fetch(session, query)
df3

CREATE TABLE IF NOT EXISTS song_listened_by_users (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))
===== Query: =======

    SELECT firstName, lastName 
    FROM song_listened_by_users 
    WHERE song='All Hands Against His Own'



,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


#### Drop the tables before closing out the sessions

In [13]:
for table in [table_1, table_2, table_3]:
    drop_table(session, table)


#### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()

TODO:
performance improvement